In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [20]:
def img_show(img):
    plt.figure("Image") # 图像窗口名称
    plt.imshow(img)
    plt.axis('off') # 关掉坐标轴为 off
    plt.show()

In [21]:
img = cv2.imread("./imgsrc/test.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,img_bin=cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
img_dst = cv2.medianBlur(img_gray, 7)

In [22]:
circle = cv2.HoughCircles(img_dst, cv2.HOUGH_GRADIENT, 1, 50,
                         param1=100, param2=100, minRadius=10, maxRadius=500)

# 将检测结果绘制在图像上
for i in circle[0, :]:  # 遍历矩阵的每一行的数据
    # 绘制圆形
    # cv2.circle(img_dst, (int(i[0]), int(i[1])), int(i[2]), (255, 0, 0), 10)
    # 绘制圆心
    cv2.circle(img, (int(i[0]), int(i[1])), 10, (255, 0, 0), -1)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
img_show(img)

In [ ]:
# P = [[458.654, 0, 367.215],
#      [0, 457.296, 248.375],
#      [0, 0, 1]]
# K = [-0.28340811, 0.07395907, 0.00019359, 1.76187114e-05]
# img_distort = cv2.undistort(img, np.array(P), np.array(K))
# img_diff = cv2.absdiff(img, img_distort)